In [1]:
import pandas as pd

In [2]:
%%time
df_merged = pd.read_csv(fr"C:\Users\User\Desktop\Final Year Project\Code\Processed_Both\merged_CX_HKO_v3_1.csv")
df_aws = pd.read_csv("C:\\Users\\User\\Desktop\\Final Year Project\\Code\\Processed_HKO\\HKO_aws_concatenated_data.csv")

<timed exec>:1: DtypeWarning: Columns (0,25,26,27,29,30,31,32,34,35,36,37,38,40,41) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 1min 34s
Wall time: 1min 35s


In [3]:
%%time
# Convert the time columns to datetime
df_merged['adjusted landing time'] = pd.to_datetime(df_merged['adjusted landing time'])
df_aws['datetime'] = pd.to_datetime(df_aws['datetime'])

CPU times: total: 2.91 s
Wall time: 2.88 s


In [4]:
%%time
# Sort the DataFrames by their respective time columns
df_merged = df_merged.sort_values('adjusted landing time')
df_aws = df_aws.sort_values('datetime')

CPU times: total: 14.2 s
Wall time: 14.3 s


In [5]:
# AWS 
# Define a dictionary with your dataframes and their corresponding prefixes
dfs = {'aws': df_aws}

tolerance_level = pd.Timedelta(minutes=15)  

for prefix, df in dfs.items():
    # Add prefix to each column name including 'DATETIME', but only if it doesn't already exist
    df.columns = [f'{prefix}_{col}' if not col.startswith(f'{prefix}_') else col for col in df.columns]
    
    df_merged = pd.merge_asof(left=df_merged, right=df,
                              left_on='adjusted landing time',
                              right_on=f'{prefix}_datetime',
                              tolerance=tolerance_level,
                              direction='backward')


In [6]:
df_merged

,Offset,UTC time,Airframe,Corrected Latitude (deg),Corrected Longitude (deg),Heading (true) (deg),Wind Direction (true) (deg),Wind Speed (knots),Airspeed (calibrated; 1 or Only) (knots),Baro-Corrected Altitude (ft),...,aws_R3E_y1,aws_R3E_wg2,aws_R3W_wg2,aws_CT5_J1,aws_CT5_L1,aws_CT5_M1,aws_CT5_N1,aws_CT5_O1,aws_CT5_P1,aws_CT5_Q1
0,NaN,5:27:03,88,22.382524,114.164636,251.455093,244.687500,11.000,175.750,4015.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8740,10:50:18,57,22.299927,113.863953,72.597664,82.968750,15.625,130.125,640.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8741,NaN,57,22.300098,113.864513,72.949226,82.265625,15.375,130.000,636.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8742,NaN,57,22.300268,113.865072,72.773445,81.562500,16.250,130.750,624.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8743,10:50:26,57,22.300439,113.865631,72.773445,81.562500,15.625,129.500,620.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754548,8013,NaN,113,22.326550,113.897753,NaN,87.187500,0.000,53.125,16.0,...,120.0,NaN,NaN,251.0,227.0,87.0,251.0,221.0,808.0,401.0
754549,8014,NaN,113,22.326446,113.897495,NaN,87.187500,0.000,50.500,24.0,...,120.0,NaN,NaN,251.0,227.0,87.0,251.0,221.0,808.0,401.0
754550,8015,8:9:22,113,22.326344,113.897278,241.786926,87.187500,0.000,48.500,28.0,...,120.0,NaN,NaN,251.0,227.0,87.0,251.0,221.0,808.0,401.0
754551,8016,NaN,113,22.326242,113.897061,NaN,87.187500,0.000,47.750,28.0,...,120.0,NaN,NaN,251.0,227.0,87.0,251.0,221.0,808.0,401.0


In [7]:
# Count the number of NaN values in each column
df_merged.isnull().sum()

Offset                         4984
UTC time                     290964
Airframe                          0
Corrected Latitude (deg)          0
Corrected Longitude (deg)         0
                              ...  
aws_CT5_M1                   724645
aws_CT5_N1                   724645
aws_CT5_O1                   724645
aws_CT5_P1                   724645
aws_CT5_Q1                   724645
Length: 768, dtype: int64

In [9]:
for column in df_merged.columns:
    missing_values = df_merged[column].isnull().sum()
    print(f"{column}: {missing_values}")

Offset: 4984
UTC time: 290964
Airframe: 0
Corrected Latitude (deg): 0
Corrected Longitude (deg): 0
Heading (true) (deg): 62840
Wind Direction (true) (deg): 8011
Wind Speed (knots): 5976
Airspeed (calibrated; 1 or Only) (knots): 0
Baro-Corrected Altitude (ft): 63961
Radio Altitude (1, left, Capt or Only) (ft): 55393
Vertical Speed (inertial) (ft/min): 0
Acceleration (normal load-factor) (g's): 0
Angle of Attack (Indicated) (LH or only) (deg): 244972
Angle of Attack (Corrected) (LH or only) (deg): 0
Pitch Attitude (Capt or Only) (deg): 0
Airport and Runway (Landing): 0
Date-Time at Touchdown: 6760
Filename: 0
adjusted landing time: 0
Angle of Attack (Best Available) (deg): 748155
predictive or reactive wind shear warning: 0
BOOL Angle of Attack (Indicated) (LH or only) (deg): 0
label: 0
15 knots at the point of GA: 0
aware_DATETIME: 643240
aware_CORRIDOR_3RS: 643240
aware_CORRIDOR_2RS: 646107
aware_MAGNITUDE: 643240
aware_LOCATION: 643240
lidar_DATETIME: 265478
lidar_CORRIDOR_3RS: 265478

In [8]:
# Save the merged result back to a new CSV file, if needed
df_merged.to_csv(r'C:\Users\User\Desktop\Final Year Project\Code\Processed_Both\merged_CX_HKO_v3_2.csv', index=False)